In [91]:
# 1. Configuration et connexion à PostgreSQL
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, func ,select, MetaData, Table, Column, Integer, String, DateTime, ForeignKey, Numeric

# Charger les variables d'environnement
load_dotenv()

# Récupérer les paramètres de connexion
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Créer l'URL de connexion
DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Créer le moteur SQLAlchemy
engine = create_engine(DATABASE_URL, echo=True)

# Créer une connexion persistante
connection = None
try:
    connection = engine.connect()
    
    print(f"✅ Connexion réussie ")
except Exception as e:
    print(f"❌ Erreur de connexion : {e}")
    connection = None

2025-07-09 16:36:29,103 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-07-09 16:36:29,103 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-09 16:36:29,105 INFO sqlalchemy.engine.Engine select current_schema()
2025-07-09 16:36:29,106 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-09 16:36:29,108 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-07-09 16:36:29,108 INFO sqlalchemy.engine.Engine [raw sql] {}
✅ Connexion réussie 


In [92]:
# 2. Définition des tables avec SQLAlchemy
metadata = MetaData()

# Table clients
clients = Table(
    'clients', metadata,
    Column('client_id', Integer, primary_key=True),
    Column('first_name', String(50), nullable=False),
    Column('last_name', String(50), nullable=False),
    Column('email', String(100), nullable=False, unique=True),
    Column('phone_number', String(15), nullable=True)
)

# Table destinations
destinations = Table(
    'destinations', metadata,
    Column('destination_id', Integer, primary_key=True),
    Column('name', String(100), nullable=False),
    Column('country', String(100), nullable=True),
    Column('price_per_person', Numeric(10, 2), nullable=False)
)

# Table bookings
bookings = Table(
    'bookings', metadata,
    Column('booking_id', Integer, primary_key=True),
    Column('client_id', Integer, ForeignKey('clients.client_id'), nullable=False),
    Column('booking_date', DateTime, nullable=False),
    Column('total_price', Numeric(10, 2), nullable=False)
)

# Table booking_items
booking_items = Table(
    'booking_items', metadata,
    Column('item_id', Integer, primary_key=True),
    Column('booking_id', Integer, ForeignKey('bookings.booking_id'), nullable=False),
    Column('destination_id', Integer, ForeignKey('destinations.destination_id'), nullable=False),
    Column('travelers_count', Integer, nullable=False)
)


In [93]:
# 3. Création des tables dans la base de données
def create_tables():
    if engine is not None:
        metadata.create_all(engine)
        print("Tables créées avec succès dans PostgreSQL.")
    else:
        print("Connexion absente, impossible de créer les tables.")

# Exécuter la création des tables
create_tables()

2025-07-09 16:36:29,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-09 16:36:29,188 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-07-09 16:36:29,189 INFO sqlalchemy.engine.Engine [generated in 0.00096s] {'table_name': 'clients', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-07-09 16:36:29,196 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_c

In [94]:
# 4. Préparation des données à insérer

# Données pour la table clients
insert_clients = clients.insert().values([
    {'first_name': 'ilham', 'last_name': 'elgharbi', 'email': 'ilham.elgharbi@example.com'},
    {'first_name': 'John', 'last_name': 'Doe', 'email': 'john.doe@example.com'},
    {'first_name': 'Alice', 'last_name': 'Smith', 'email': 'alice.smith@example.com'},
    {'first_name': 'Bob', 'last_name': 'Johnson', 'email': 'bob.johnson@example.com'},
    {'first_name': 'Charlie', 'last_name': 'Williams', 'email': 'charlie.williams@example.com'}
])

# Données pour la table destinations
insert_destinations = destinations.insert().values([
    {'name': 'Paris', 'country': 'France', 'price_per_person': 1500.00},
    {'name': 'New York', 'country': 'USA', 'price_per_person': 2000.00},
    {'name': 'Tokyo', 'country': 'Japan', 'price_per_person': 2500.00},
    {'name': 'Sydney', 'country': 'Australia', 'price_per_person': 3000.00},
    {'name': 'Cape Town', 'country': 'South Africa', 'price_per_person': 1800.00}
])

# Données pour la table bookings
insert_bookings = bookings.insert().values([
    {'client_id': 1, 'booking_date': '2023-10-01 10:00:00', 'total_price': 1500.00},
    {'client_id': 2, 'booking_date': '2023-10-02 11:00:00', 'total_price': 2000.00},
    {'client_id': 3, 'booking_date': '2023-10-03 12:00:00', 'total_price': 2500.00},
])

# Données pour la table booking_items
insert_booking_items = booking_items.insert().values([
    {'booking_id': 1, 'destination_id': 1, 'travelers_count': 2},
    {'booking_id': 2, 'destination_id': 2, 'travelers_count': 3},
    {'booking_id': 3, 'destination_id': 3, 'travelers_count': 1},
])


In [95]:
def execute_insert_statements():
    if connection is not None:
        try:
            # Execute each insert statement
            result_clients = connection.execute(insert_clients)
        
            
            result_destinations = connection.execute(insert_destinations)
        
            
            result_bookings = connection.execute(insert_bookings)
        
            
            result_booking_items = connection.execute(insert_booking_items)
        
            
            # Commit the transaction to make changes permanent
            connection.commit()
            print("Toutes les données ont été insérées avec succès!")
            
        except Exception as e:
            print(f"Erreur lors de l'insertion des données: {e}")
            connection.rollback()
    else:
        print("Connexion absente, impossible d'insérer les données.")


In [96]:
# 6. Fonction pour afficher les clients
def afficher_clients():
    if connection is not None:
        try:
            # Create the SELECT statement
            select_clients = clients.select()
            # Execute the statement using connection.execute()
            result = connection.execute(select_clients)
            # Iterate over the actual results
            print("=== Liste des Clients ===")
            for row in result:
                print(f"ID: {row.client_id}, Nom: {row.first_name} {row.last_name}, Email: {row.email}, Téléphone: {row.phone_number}")
        except Exception as e:
            print(f"Erreur lors de l'affichage des clients: {e}")
    else:
        print("Connexion absente, impossible d'afficher les clients.")
afficher_clients()

2025-07-09 16:36:29,244 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-09 16:36:29,245 INFO sqlalchemy.engine.Engine SELECT clients.client_id, clients.first_name, clients.last_name, clients.email, clients.phone_number 
FROM clients
2025-07-09 16:36:29,245 INFO sqlalchemy.engine.Engine [generated in 0.00210s] {}
=== Liste des Clients ===
ID: 2, Nom: John Doe, Email: john.doe@example.com, Téléphone: None
ID: 3, Nom: Alice Smith, Email: alice.smith@example.com, Téléphone: None
ID: 4, Nom: Bob Johnson, Email: bob.johnson@example.com, Téléphone: None
ID: 5, Nom: Charlie Williams, Email: charlie.williams@example.com, Téléphone: None
ID: 1, Nom: ilham elgharbi, Email: kok@gmail.com, Téléphone: None


In [97]:
# 7. Fonction pour afficher les destinations (avec filtre prix > 1000€)
def afficher_destinations():
    if connection is not None:
        try:
            # Create the SELECT statement
            select_destinations = destinations.select()
            # Execute the statement using connection.execute()
            result = connection.execute(select_destinations)
            # Iterate over the actual results
            print("=== Liste des Destinations (Prix > 1000€) ===")
            for row in result:
                if row.price_per_person > 1000:
                    print(f"ID: {row.destination_id}, Nom: {row.name}, Pays: {row.country}, Prix: {row.price_per_person}€")
        except Exception as e:
            print(f"Erreur lors de l'affichage des destinations: {e}")
    else:
        print("Connexion absente, impossible d'afficher les destinations.")

afficher_destinations()

2025-07-09 16:36:29,259 INFO sqlalchemy.engine.Engine SELECT destinations.destination_id, destinations.name, destinations.country, destinations.price_per_person 
FROM destinations
2025-07-09 16:36:29,261 INFO sqlalchemy.engine.Engine [generated in 0.00217s] {}
=== Liste des Destinations (Prix > 1000€) ===
ID: 2, Nom: New York, Pays: USA, Prix: 2000.00€
ID: 3, Nom: Tokyo, Pays: Japan, Prix: 2500.00€
ID: 4, Nom: Sydney, Pays: Australia, Prix: 3000.00€
ID: 5, Nom: Cape Town, Pays: South Africa, Prix: 1800.00€
ID: 1, Nom: Paris, Pays: France, Prix: 4279.69€


In [98]:
# 8. Fonction pour afficher les réservations avec détails client
def afficher_bookings():
    if connection is not None:
        try:
            # Join bookings with clients to get client details
            query = bookings.join(clients).select()
            result = connection.execute(query)
            
            print("=== Liste des Réservations ===")
            for row in result:
                print(f"Booking ID: {row.booking_id}, Client: {row.first_name} {row.last_name}, Date: {row.booking_date}, Prix: {row.total_price}€")
        except Exception as e:
            print(f"Erreur lors de l'affichage des réservations: {e}")
    else:
        print("Connexion absente, impossible d'afficher les réservations.")

# Afficher les réservations
afficher_bookings()

2025-07-09 16:36:29,274 INFO sqlalchemy.engine.Engine SELECT bookings.booking_id, bookings.client_id, bookings.booking_date, bookings.total_price, clients.client_id AS client_id_1, clients.first_name, clients.last_name, clients.email, clients.phone_number 
FROM bookings JOIN clients ON clients.client_id = bookings.client_id
2025-07-09 16:36:29,276 INFO sqlalchemy.engine.Engine [generated in 0.00143s] {}
=== Liste des Réservations ===
Booking ID: 1, Client: ilham elgharbi, Date: 2023-10-01 10:00:00, Prix: 1500.00€
Booking ID: 2, Client: John Doe, Date: 2023-10-02 11:00:00, Prix: 2000.00€
Booking ID: 3, Client: Alice Smith, Date: 2023-10-03 12:00:00, Prix: 2500.00€


In [99]:
update_clients = clients.update().where(clients.c.client_id == 1).values(
    email='kok@gmail.com'
)

result = connection.execute(update_clients)
connection.commit()
print("✅ Email mis à jour avec succès.")



2025-07-09 16:36:29,291 INFO sqlalchemy.engine.Engine UPDATE clients SET email=%(email)s WHERE clients.client_id = %(client_id_1)s
2025-07-09 16:36:29,292 INFO sqlalchemy.engine.Engine [generated in 0.00127s] {'email': 'kok@gmail.com', 'client_id_1': 1}
2025-07-09 16:36:29,295 INFO sqlalchemy.engine.Engine COMMIT
✅ Email mis à jour avec succès.


In [100]:
update_prix = destinations.update().where(destinations.c.country == 'France').values(
    price_per_person = destinations.c.price_per_person * 1.10
)

result = connection.execute(update_prix)
connection.commit()
print("✅ Prix mis à jour avec succès.")


2025-07-09 16:36:29,306 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-09 16:36:29,307 INFO sqlalchemy.engine.Engine UPDATE destinations SET price_per_person=(destinations.price_per_person * %(price_per_person_1)s) WHERE destinations.country = %(country_1)s
2025-07-09 16:36:29,308 INFO sqlalchemy.engine.Engine [generated in 0.00216s] {'price_per_person_1': 1.1, 'country_1': 'France'}
2025-07-09 16:36:29,311 INFO sqlalchemy.engine.Engine COMMIT
✅ Prix mis à jour avec succès.


In [101]:
nb_clients = connection.execute(select(func.count(clients.c.client_id))).scalar()
print(f"\nNombre total de clients: {nb_clients}")

2025-07-09 16:36:29,321 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-09 16:36:29,322 INFO sqlalchemy.engine.Engine SELECT count(clients.client_id) AS count_1 
FROM clients
2025-07-09 16:36:29,323 INFO sqlalchemy.engine.Engine [generated in 0.00208s] {}

Nombre total de clients: 5


In [102]:
avg_price = connection.execute(select(func.avg(destinations.c.price_per_person))).scalar()
print(f"Prix moyen des destinations: {avg_price:.2f} €")

2025-07-09 16:36:29,334 INFO sqlalchemy.engine.Engine SELECT avg(destinations.price_per_person) AS avg_1 
FROM destinations
2025-07-09 16:36:29,335 INFO sqlalchemy.engine.Engine [generated in 0.00103s] {}
Prix moyen des destinations: 2801.53 €


In [103]:
travelers_per_dest = connection.execute(select(booking_items.c.destination_id, func.sum(booking_items.c.travelers_count)).group_by(booking_items.c.destination_id))
print("Total voyageurs par destination:")
for row in travelers_per_dest:
    print(row)

2025-07-09 16:36:29,346 INFO sqlalchemy.engine.Engine SELECT booking_items.destination_id, sum(booking_items.travelers_count) AS sum_1 
FROM booking_items GROUP BY booking_items.destination_id
2025-07-09 16:36:29,347 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {}
Total voyageurs par destination:
(3, 1)
(2, 3)
(1, 2)


In [104]:
print("\nDestinations réservées > 2 fois:")
res = connection.execute(select(booking_items.c.destination_id, func.count()).group_by(booking_items.c.destination_id).having(func.count() > 2))
if res.rowcount == 0:
    print("Aucune destination réservée plus de 2 fois.")
else :
        for row in res:
                print(row)



Destinations réservées > 2 fois:
2025-07-09 16:36:29,359 INFO sqlalchemy.engine.Engine SELECT booking_items.destination_id, count(*) AS count_1 
FROM booking_items GROUP BY booking_items.destination_id 
HAVING count(*) > %(count_2)s
2025-07-09 16:36:29,360 INFO sqlalchemy.engine.Engine [generated in 0.00090s] {'count_2': 2}
Aucune destination réservée plus de 2 fois.


In [105]:
print("\nClients avec > 5 voyageurs:")
join_all = booking_items.join(bookings, booking_items.c.booking_id == bookings.c.booking_id)
res = connection.execute(select(bookings.c.client_id, func.sum(booking_items.c.travelers_count)).select_from(join_all).group_by(bookings.c.client_id).having(func.sum(booking_items.c.travelers_count) > 5))
if res.rowcount == 0:
    print("Aucun client avec plus de 5 voyageurs.")
else :
    print("Clients avec plus de 5 voyageurs:")
    for row in res:
        print(row)


Clients avec > 5 voyageurs:
2025-07-09 16:36:29,371 INFO sqlalchemy.engine.Engine SELECT bookings.client_id, sum(booking_items.travelers_count) AS sum_1 
FROM booking_items JOIN bookings ON booking_items.booking_id = bookings.booking_id GROUP BY bookings.client_id 
HAVING sum(booking_items.travelers_count) > %(sum_2)s
2025-07-09 16:36:29,372 INFO sqlalchemy.engine.Engine [generated in 0.00077s] {'sum_2': 5}
Aucun client avec plus de 5 voyageurs.
